In [129]:
### Load libraries
library(here)
library(rgeos)
library(geosphere)
library(rgdal)
library("ggplot2")
library("tidyr")
library(raster) # for crs
library(ggbeeswarm)
library(tmap)
library(sf)
library(stringr)
library(scales)
library("ggrepel")

In [2]:
### Define paths

# Define imputs paths:

# Admnistrative map of China
path2china<-here('raw_data','CHN_adm')
path2out<-here("outputs")

# Archaeological records location data
path2buckwheat<-here("data","ESM_2.csv") # Buckwheat's occurrence records
path2locations<-here("data","site_locations.csv")

# Production and predictions data
path2predictions <- here("outputs","predictions")

In [86]:
# Define output paths:

path2outputs<-here("manuscript","Chapter7","Figs","Raster")

In [3]:
crop="millet"

In [143]:
rivers<-st_read(dsn=here("raw_data","ne_50m_rivers_lake_centerlines"),layer='ne_50m_rivers_lake_centerlines')
rivers2<-st_read(dsn=here("raw_data","ne_110m_rivers_lake_centerlines"),layer='ne_110m_rivers_lake_centerlines')



Reading layer `ne_50m_rivers_lake_centerlines' from data source `G:\My Drive\my_repositories\PhD\raw_data\ne_50m_rivers_lake_centerlines' using driver `ESRI Shapefile'
Simple feature collection with 478 features and 36 fields (with 1 geometry empty)
geometry type:  MULTILINESTRING
dimension:      XY
bbox:           xmin: -165.2439 ymin: -50.24014 xmax: 176.3258 ymax: 73.3349
geographic CRS: WGS 84
Reading layer `ne_110m_rivers_lake_centerlines' from data source `G:\My Drive\my_repositories\PhD\raw_data\ne_110m_rivers_lake_centerlines' using driver `ESRI Shapefile'
Simple feature collection with 13 features and 35 fields
geometry type:  LINESTRING
dimension:      XY
bbox:           xmin: -135.3134 ymin: -33.99358 xmax: 129.956 ymax: 72.90651
geographic CRS: WGS 84


In [ ]:
outer_border<-st_read(dsn=path2china,layer='CHN_adm0')
posterior <-read.csv(paste(path2predictions,"/",crop,"_posterior_predictions_summary.csv",sep=""))

In [ ]:
rivers<-st_read(dsn=path2china,layer='CHN_adm1')

In [6]:
# Import the outer border of China
outer_border<-st_read(dsn=path2china,layer='CHN_adm0')
# Import the spatial polygons data frame of counties:
china<-readOGR(dsn=path2china,layer='CHN_adm3')
# Import production data

Reading layer `CHN_adm0' from data source `G:\My Drive\my_repositories\PhD\raw_data\CHN_adm' using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 70 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: 73.5577 ymin: 18.15931 xmax: 134.7739 ymax: 53.56086
geographic CRS: WGS 84
OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\my_repositories\PhD\raw_data\CHN_adm", layer: "CHN_adm3"
with 2409 features
It has 13 fields
Integer64 fields read as strings:  ID_0 ID_1 ID_2 ID_3 


In [7]:
str(china@data)

'data.frame':	2409 obs. of  13 variables:
 $ ID_0     : chr  "49" "49" "49" "49" ...
 $ ISO      : chr  "CHN" "CHN" "CHN" "CHN" ...
 $ NAME_0   : chr  "China" "China" "China" "China" ...
 $ ID_1     : chr  "1" "1" "1" "1" ...
 $ NAME_1   : chr  "Anhui" "Anhui" "Anhui" "Anhui" ...
 $ ID_2     : chr  "1" "1" "1" "1" ...
 $ NAME_2   : chr  "Anqing" "Anqing" "Anqing" "Anqing" ...
 $ ID_3     : chr  "1" "2" "3" "4" ...
 $ NAME_3   : chr  "Anqing" "Huaining" "Qianshan" "Susong" ...
 $ TYPE_3   : chr  "XiÃ njÃ­shÃ¬" "XiÃ n" "XiÃ n" "XiÃ n" ...
 $ ENGTYPE_3: chr  "County City" "County" "County" "County" ...
 $ NL_NAME_3: chr  NA "æ\200\200å®\201åŽ¿" "æ½œå±±åŽ¿" "å®¿æ\235¾åŽ¿" ...
 $ VARNAME_3: chr  NA "HuÃ¡inÃ­ng" "QiÃ¡nshÄ\201n" "SÃ¹sÅ\215ng" ...


In [8]:
crop<-"millet"

In [9]:
 print(crop)
    path2pred_summary<-paste(path2predictions,"/",crop,"_predictions.csv",sep="")
    pred_summary<-read.csv(path2pred_summary,row.names=1)

    ### Get the log means predictions, 0.05, and 0.95 quantiles for each county in a separate variables
    mean_names<-colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("mean",colnames(pred_summary))]
    mean<-pred_summary[,colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("mean",colnames(pred_summary))]]
    q5_names<-colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X5",colnames(pred_summary))]
    q5<-pred_summary[,colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X5",colnames(pred_summary))]]
    q95_names<-colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X95",colnames(pred_summary))]
    q95<-pred_summary[,colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X95",colnames(pred_summary))]]

    # Put the values together in the dataframe
    df<-cbind(mean,q5,q95)
    colnames(df)<-paste(crop,colnames(df),sep="_")
    # Simplify colum names a bit
    colnames(df)<-gsub("s_pred","",colnames(df))
    colnames(df)<-gsub("_X","",colnames(df))
    colnames(df)<-gsub("\\.","",colnames(df))
    
    # Bind data to spatial polygons
    china@data<-cbind(china@data,df)

[1] "millet"


In [13]:
locdata<-read.csv(here("data","locations_suitability.csv"))

In [19]:
threshold<-min(locdata[,paste(crop,"Mean",sep="")],na.rm=TRUE)

In [26]:
columns<-c("millet_mean_6000_log","millet_mean_5000_log")

In [47]:
min<-floor(min(c(min(china@data[columns]),posterior$X5._log)))
max<-ceiling(max(c(max(china@data[columns])),posterior$X95._log))

In [179]:
pred_maps<-tm_shape(china,bbox=c(108, 15, 125, 35))+
    # Define the grid form maps drawing
    tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=3,labels.size = 0.8)+
    # Define the colors of the polygons and the legend
    tm_fill(col=columns,style="cont",breaks=seq(min,max,5),legend.reverse = TRUE,palette="-RdYlBu",title = paste("log area\nfraction\nunder\n",crop,"\ncultivation",sep=""),showNA = F,midpoint=threshold)+
    tm_facets(free.scales=FALSE,free.coords=FALSE,ncol = 2)+    
    tm_shape(rivers)+
    tm_lines(col="steelblue",lwd=1.2)+
    tm_shape(rivers2)+
    tm_lines(col="steelblue",lwd=1.2)+
    tm_shape(outer_border)+
    tm_borders(col="grey35",lwd=1)+
    tm_text(text="NAME_1",size=0.6,col="grey35")+
    tm_layout(panel.labels=c("6000 BP","5000 BP"),
              panel.label.bg.color="white",
              frame=FALSE,
              legend.outside.size=0.08,
              legend.outside = TRUE,
              legend.text.size = 0.4,
              panel.label.size = 1,
              panel.label.fontface = "bold",
              legend.title.size=0.8,
              frame.lwd = NA,
              panel.label.height=2,
              attr.outside=TRUE,
              outer.margins=c(0.00001,0.01,0.01,0.01)
             )

In [ ]:
width=14.3
height=8

tmap_save(tm = pred_maps,filename = paste(path2outputs,"\\","southChina.png",sep=""),scale=1,width = width,height = height,units = "cm",dpi = 300)

In [172]:
pred_maps<-tm_shape(china,bbox=c(108, 15, 125, 35))+
    # Define the grid form maps drawing
    tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=3,labels.size = 0.8)+
    # Define the colors of the polygons and the legend
    tm_fill(col=columns,col.title=c("1","2"),style="cont",breaks=seq(min,max,5),legend.reverse = TRUE,palette="-RdYlBu",title = paste("log area\nfraction\nunder\n",crop,"\ncultivation",sep=""),showNA = F,midpoint=threshold)+
    tm_facets(free.scales=FALSE,free.coords=FALSE,ncol = 2)+    
    tm_shape(rivers)+
    tm_lines(col="steelblue",lwd=1.2)+
    tm_shape(rivers2)+
    tm_lines(col="steelblue",lwd=1.2)+
    tm_shape(outer_border)+
    tm_borders(col="black",lwd=1)+
    tm_text(text="NAME_1",size=0.6)